In [481]:
import snappy
import ast
import csv
import pickle
from ast import literal_eval

Let's implement functions which change a pd-code by a crossing change, and recognize the sign of a crossing:

In [482]:
def cc(pd2,i):
    '''
    changes the ith crossing
    '''
    pd=pd2.copy()
    if pos_crossing(pd,i):
        [a,b,c,d]=pd[i]
        pd[i]=[d,a,b,c]
    else:
        [a,b,c,d]=pd[i]
        pd[i]=[b,c,d,a]
    return(pd)

def pos_crossing(pd,i):
    '''
    True if the i-th crossing is positive, False else
    '''
    n=max([max(k) for k in pd])
    (a,b,c,d)=pd[i]
    if d==n:
        if b==0:
            return(True)
    if b==n:
        if d==0:
            return(False)
    if d<b:
        return(True)
    return(False)

Lets implement functions which take a pd code and return a list of pd codes corresponding to: 
1) all the diagrams obtained by changing a crossing

2) all diagrams obtained by changing a positive and a negative crossing

In [483]:
def crossing_changer(pd):
    '''
    input: A pd-code
    output: list of pd codes corresponding crossing changes in pd
    '''
    l=[]
    for i in range(len(pd)):
        new_pd=cc(pd,i)
        l.append([new_pd,[i,'cc']])
    return(l)


def two_crossing_changer(pd):
    '''
    input: pd code
    output: returns list of pd codes corresponding to changement of a positive and a negative crossing
    '''
    l=[]
    for i in range(len(pd)):
        if pos_crossing(pd,i):
            for j in range(len(pd)):
                if not pos_crossing(pd,j) and i!=j:
                    l.append([cc(cc(pd,i),j),[(i,j),'cc+-']])
    return(l)

Lets get the knotinfo data of the slice genera of prime knots with less than 13 crossings

In [484]:
name_slice_knotinfo=[]
with open('/Users/leomousseau/Desktop/Bachelor code census/slice_knotinfo','r') as inp:
    reader=csv.reader(inp)
    for row in reader:
        name_slice_knotinfo.append([row[0],int(row[1])])
slice_genus_knotinfo=dict(name_slice_knotinfo)
        
name_top_knotinfo=[]
with open('/Users/leomousseau/Desktop/Bachelor code census/top_knotinfo','r') as inp:
    reader=csv.reader(inp)
    for row in reader:
        try:
            name_top_knotinfo.append([row[0],int(row[1])])
        except:
            name_top_knotinfo.append([row[0],2]) # those are all (1,2) and we bound them by 2 from above for our purposes later
topological_genus_knotinfo=dict(name_top_knotinfo)

Lets implement a function obtaining the prime factors from a given knot

In [485]:
def prime_decomposition(K,decomp=[]):
    '''
    input: a Knot K
    output: a list of knots corresponding to the prime factors of K
    '''
    
    
    # doesnt work perfectly since the implemented function deconnect sum is not perfec
    deconnect=K.deconnect_sum()
    if len(deconnect)==1: #i.e. K is prime
        decomp.append(K)
        return(decomp)
 
    for L in deconnect:
        decomp=decomp+prime_decomposition(L,[])
    
    return(decomp)

Since SnapPy does not recognize some torus knots, we implement a function which does determine if a given diagram represents a torus knot, and if it does, it returns its coefficients (p,q)

In [486]:
def identify_torus(pd):
    '''
    input: a knot
    output: if the output is (p,q), then the knot is the p,q torus knot. If the return is false, then it is not
    '''
    K=snappy.Link(pd)
    if len(K.deconnect_sum())!=1:
        return(['False','non-prime!'])
    # therefore K is prime, and determined by its knot group (by rigidity)
    g=K.exterior().fundamental_group()
    #print(g)
    if g.generators()==['a', 'b']:
        if len(g.relators())==1:
            word=g.relators()[0]
            counter=0
            for i in range(len(word)):
                if word[i]=='a' or word[i]=='A':
                    if i!=len(word)-1:
                        if word[i+1]=='b' or word[i+1]=='B':
                            counter=counter+1
            if counter==1: #its torus
                p=word.count('a')
                q=word.count('b')
                return(['True',(p,q)])
            
            if counter!=1:
                return(['False','relator word not right'])
        else:
            return(['False','relator'])
    else:
        return(['False','generators'])



Lets implement a function which can identify a low crossing knot and return bounds on its slice genera

In [487]:
def low_crossing_genus(pd):
    '''
    input: pd code
    output: bound on both slice genera
    '''
    K=snappy.Link(pd)
    M=K.exterior()
    if len(M.fundamental_group().generators())==1 and M.fundamental_group().relators()==[]:
        return((0,0))
    name='undefined'
    m=0
    if name=='undefined' and m<100:
        if M.identify()==[]:
            M.randomize()
        else:
            name=M.identify()[0].name()
        m=m+1
    if name!='undefined':
        slice_genus=slice_genus_knotinfo[name]
        topological_genus=topological_genus_knotinfo[name]
        slice_bound=slice_genus
        top_bound=topological_genus
    else: #the knot is composite
        prime_decomp=prime_decomposition(K)
        slice_sum=0
        top_sum=0
        for L in prime_decomp:
            M=L.exterior()
            name='undefined'
            m=0
            if name=='undefined' and m<100:
                if M.identify()==[]:
                    M.randomize()
                else:
                    name=M.identify()[0].name()
                m=m+1
            try:
                slice_sum+=slice_genus_knotinfo[name]
                top_sum+=topological_genus_knotinfo[name]
            except:
                # this error occurs when the knot is torus, since SnapPy cannot identify them because 
                # they are not hyperbolic. We write a function which recognizes torus knots, and output 
                # the coefficients (p,q) which allow us to get 
                [x,y]=identify_torus(L.PD_code())
                if x=='True':
                    (p,q)=y
                    slice_sum+=(1/2)*(p-1)*(q-1)
                    top_sum+=(1/2)*(p-1)*(q-1)
                else:
                    print(L.PD_code(),'ERROR, it is not identifiable and not torus')
                    slice_sum+=5 #'largest slice genus among low crossing knots'
                    top_sum+=5 #'largest slice genus among low crossing knots'
            
        slice_bound=slice_sum
        top_bound=top_sum
        
    return(slice_bound,top_bound)    

The following function takes a pd_code, and returns an upper bound on the slice- and topological genus of the knot. Its does this by applying the two first operations from Lemma x to the pd code, and then simplifies each knot. If the simplified pd codes all have more than 12 crossings it repeats the process with the pd code of a knot with the least crossings. If there are simplified pd codes with less than 13 crossings, we get their slice and topological genus (or an upper bound on them using proposition x if the knots are not prime) from knotinfo. Among those, we look which of them have the lowest slice- and topological genus and return the respective slice genera plus the number of operations (counter) it took to obtain the knots. Throughout the process we save exactly which operations have been applied, so we can verify the bounds again if we wish to.

In [488]:
def upper_bounds(pd4,counter=0,change_sequence=[]):
    '''
    input: A PD-code of a knot K
    output:[(g1,g2),change_sequence] where g1 is an upper bound on the slice genus
            of K, and g2 is an upper bound on the topological genus of K. 
            Change_sequence is a list with each element a PD-code together with
            the precise operation we applied to it to get to the PD-code in
            the next element of the list
    '''
    pd=pd4.copy()
    counter+=1
    new_pds1=two_crossing_changer(pd)+crossing_changer(pd)      #+resoluter(pd)
    new_pds2=[]
    for k in range(len(new_pds1)):
        t=new_pds1[k]         # t=[pd,[(i,j),'cc+-']] or t=[pd,[i,'cc']]
        pd_new=t[0]
        K=snappy.Link(pd_new)
        K.simplify('level')  # one can also simplify more by replacing "level" with "global"
        simplified_pd=K.PD_code()
        new_pds2.append([simplified_pd,len(simplified_pd),t[1]])   
    n=min([j[1] for j in new_pds2])        
    if n<13:  # we have reached a low crossing knot
        G=[]  #now lets look which low crossing knot has the lowest slice genus! 
        for [new_pd,c,change] in new_pds2:
            if c<13:    
                (g_slice,g_top)=low_crossing_genus(new_pd)
                G.append([new_pd,(g_slice,g_top),change])
        slice_min=min([k[1][0] for k in G]) # the minimal slice genus among all new low crossing pds
        top_min=min([k[1][1] for k in G]) # the minimal topological slice genus among all new low crossing pds
        best_slice=[]
        best_top=[]
        for k in G:
            if k[1][0]==slice_min:
                best_slice.append(k)
            if k[1][1]==top_min:
                best_top.append(k)     
        if slice_min==top_min:
            [best_pd,(g1,g2),best_change]=best_slice[0]
            change_sequence.append([pd,best_change])
            slice_bound=counter+g1
            top_bound=counter+g2
            return([(slice_bound,top_bound),change_sequence])
        else: #if the minimal topological genus is different from the minimal slice genus
            [best_pd_slice,(g1_slice,g2_slice),best_change_slice]=best_slice[0] 
            [best_pd_top,(g1_top,g2_top),best_change_top]=best_top[0]
            change_sequence.append([pd,[best_change_slice,best_change_top]])
            slice_bound=counter+g1_slice
            top_bound=counter+g2_top
            return([(slice_bound,top_bound),change_sequence])
    else:    
        best_ones=[]            #
        for j in new_pds2:
            if j[1]==n:
                best_ones.append([j[0],j[2]])  
        [best_pd,best_change]=best_ones[0] 
        change_sequence.append([pd,best_change])
        return(upper_bounds(best_pd,counter,change_sequence))

Now lets apply the final function upper bounds to all census knots who have not be determined to be strongly quasipositive

Let's first import simplified diagrams of these. These diagrams have been obtained by simplifying the diagrams obtained by taking the closure of a braid word of the census knot.

In [489]:
pd_codes_not_SQP=[]  
with open('/Users/leomousseau/Desktop/Bachelor code census/pd_codes_not_SQP','r') as inp:
    reader=csv.reader(inp)
    for row in reader:
        pd_codes_not_SQP.append(row)
for k in pd_codes_not_SQP:
    k[1]=literal_eval(k[1])
len(pd_codes_not_SQP)

497

In [ ]:
L=[]
for [name,pd] in pd_codes_not_SQP:
    [(g1,g2),change_sequence]=upper_bounds(pd,0,[])
    L.append([name,(g1,g2),change_sequence])

In [ ]:
L2=[[k[0],k[1]] for k in L]

lets us quickly add the low crossing slice genera of the census knots which already have a diagram with less than 13 crossings

In [505]:
for k in pd_codes_not_SQP:
    if len(k[1])<13:
        g1=slice_genus_knotinfo[k[0]]
        g2=topological_genus_knotinfo[k[0]]
        L2.append([k[0],(g1,g2)])

lets also add the upper bound g^top(K)=< (1/2)*degree(alexander polynomial)

In [527]:
for k in L2:
    L=snappy.Link(pd_dict[k[0]])
    p=L.exterior().alexander_polynomial()
    if k[1][1]>(1/2)*p.degree():
        k[1]=(k[1][0],(1/2)*p.degree())

Lets put these bounds together with the bounds of the SQP knots

In [538]:
all_pd_codes=[]
with open('/Users/leomousseau/Desktop/Bachelorarbeit/pd_codes_census','r') as inp:
    reader=csv.reader(inp)
    for row in reader:
        all_pd_codes.append(row)
pd_dict=dict(pd_codes)

In [540]:
pd_codes_SQP=[]
for k in all_pd_codes:
    if k[0] not in [j[0] for j in pd_codes_not_SQP]:
        pd_codes_SQP.append(k)

770

Let's get the computed Seifert genus of all census knots 

In [544]:
seifert_genus=[]
with open('/Users/leomousseau/Desktop/Bachelor code census/seifert genus census knots.csv','r') as inp:
    reader=csv.reader(inp)
    for row in reader:
        seifert_genus.append(row)
del seifert_genus[0]
for k in seifert_genus:
    k[1]=eval(k[1])
seifert_dict=dict(seifert_genus)

as discussed before, the seifert genus is equal to the slice genus for strongly quasipositive knots

In [547]:
bounds_SQP=[]

for k in pd_codes_SQP:
    slice_genus=seifert_dict[k[0]]
    p=snappy.Link(eval(k[1])).exterior().alexander_polynomial()
    top_bound=min(slice_genus,(1/2)*p.degree())
    bounds_SQP.append([k[0],(slice_genus,top_bound)])


Now combine the lists to obtain the upper bounds list of the slice genera of the census knots:

In [551]:
final_upper_bounds=bounds_SQP+L2

sort the list before saving it:

In [554]:
dict2=dict(final_upper_bounds)
final_list=[]
for k in pd_codes:
    final_list.append([k[0],dict2[k[0]]])
    

In [557]:
with open('upper_bounds_census_final.csv','w') as out:
    writer=csv.writer(out)
    for row in final_list:
        writer.writerow(row)